In [4]:
#!/usr/bin/env python
# coding: utf-8
# Autor: Ashuin Sharma (A35029)
# Tarea 3.

import os
import requests
import zipfile
import csv
import fiona
import fiona.crs
from shapely.geometry import Point, mapping, shape
from owslib.wfs import WebFeatureService
from geojson import dump
import pandas as pd

print("Descargando capas de cantones y vias..")
# Cargamos la capa de Cantones 1:5mil del SNIT
wfs = WebFeatureService(url='https://geos.snitcr.go.cr/be/IGN_5/wfs', version='1.1.0')
list(wfs.contents)

# Solicitud de capa WFS de Limite Cantonal mediante GET, para retornarse como JSON
# Parámetros de la solicitud
params = dict(service='WFS',
              version='1.1.0', 
              request='GetFeature', 
              typeName='IGN_5:limitecantonal_5k',
              srsName='urn:ogc:def:crs:EPSG::5367',
              outputFormat='json')

# Solicitud
response = requests.get("https://geos.snitcr.go.cr/be/IGN_5/wfs", params=params)

# Descarga de la respuesta en un archivo GeoJSON en cantones.geojson
with open('cantones.geojson', 'w') as file:
   dump(response.json(), file)

# Cargamos la capa de Red Vial 1:200mil del SNIT
wfs = WebFeatureService(url='https://geos.snitcr.go.cr/be/IGN_200/wfs', version='1.1.0')
list(wfs.contents)

# Solicitud de capa WFS de Limite Cantonal mediante GET, para retornarse como JSON
# Parámetros de la solicitud
params = dict(service='WFS',
              version='1.1.0', 
              request='GetFeature', 
              typeName='IGN_200:redvial_200k',
              srsName='urn:ogc:def:crs:EPSG::5367',
              outputFormat='json')

# Solicitud
response = requests.get("https://geos.snitcr.go.cr/be/IGN_200/wfs", params=params)

# Descarga de la respuesta en un archivo GeoJSON en redvial.geojson
with open('redvial.geojson', 'w') as file:
   dump(response.json(), file)

print("Creando archivo GPKG...")
# Creacion del Archivo GeoPackage
# Agregamos capa Cantones al GPKG
with fiona.open('cantones.geojson') as source:
    with fiona.open('densidad-vial.gpkg', 'w', 'GPKG', source.schema, source.crs, layer='cantones') as sink:
        for record in source:
            sink.write(record)

# Agregamos capa Red Vial al GPKG
with fiona.open('redvial.geojson') as source:
    with fiona.open('densidad-vial.gpkg', 'w', 'GPKG', source.schema, source.crs, layer='redvial') as sink:
        for record in source:
            sink.write(record)

# Conteo de longitud de red vial sobre geometria del Canton
# Esquema de la capa con el area del canton, longitud de red vial y densidad
schema = {'geometry':'Unknown',
          'properties':{
                        'rowid':'int',
                        'cod_canton':'int',
                        'canton':'str',
                        'calc_area':'float',
                        'calc_longitud':'float',
                        'calc_densidad_vial':'float',
                        'calc_longitud_autopistas':'float',
                        'calc_longitud_carr_pav_2_vias':'float',
                        'calc_longitud_carr_pav_1_vias':'float',
                        'calc_longitud_carr_nopav_2_vias':'float',
                        'calc_longitud_caminos_tierra':'float',
                       }}

# Recorremos los cantones
print("Calculando areas, longitudes e interescciones...")
with fiona.collection('densidad-vial.gpkg', 'r', layer='cantones') as cant:
    i = 1 # contador de Cantones, para imprimir el progreso del procedimiento
    # Creamos geojson de capa generada con atributos calculados
    with fiona.open('redvial-cantones.geojson','w','GeoJSON', schema, cant.crs) as sink:
        # Por cada Canton calculamos area, interseccion con vias, y la densidad
        for record_cant in cant:
            calc_area =  shape(record_cant['geometry']).area / 1000000
            cod_canton = record_cant['properties']['cod_canton']
            canton = record_cant['properties']['canton']
            calc_longitud = 0.0 # acumulador de longitud vial en el canton
            # Calculos adicionales de longitud
            calc_longitud_autopistas = 0.0 # acumulador de longitud autopistas
            calc_longitud_carr_pav_2_vias = 0.0 # acumulador longitud carreteras con pavimento a 2 o mas vias
            calc_longitud_carr_pav_1_vias = 0.0 # acumulador longitud carretaras con pavimento a 1 via
            calc_longitud_carr_nopav_2_vias = 0.0 # acumulador longitud carreteras sin pavimento a 2 vias
            calc_longitud_caminos_tierra = 0.0 # acumulador de longitud caminos de tierra
            # Calculamos la interseccion y la densidad para el canton
            with fiona.collection('densidad-vial.gpkg', 'r', layer='redvial') as redvial:    
                for registro_red in redvial:
                    intersection = shape(record_cant['geometry']).intersection(shape(registro_red['geometry']))
                    calc_longitud += intersection.length
                    categoria = registro_red['properties']['categoria']
                    match categoria:
                        case "AUTOPISTA":
                            calc_longitud_autopistas += intersection.length
                        case "CARRETERA PAVIMENTO DOS VIAS O MAS":
                            calc_longitud_carr_pav_2_vias += intersection.length
                        case "CARRETERA PAVIMENTO UNA VIA":
                            calc_longitud_carr_pav_1_vias += intersection.length
                        case "CARRETERA SIN PAVIMENTO DOS VIAS":
                            calc_longitud_carr_nopav_2_vias += intersection.length
                        case "CAMINO DE TIERRA":
                            calc_longitud_caminos_tierra += intersection.length
                        case _:
                            continue
                            
            calc_longitud = calc_longitud / 1000
            densidad = calc_longitud / calc_area
            print(i, cod_canton, canton, calc_area, calc_longitud, densidad)
            # Escribimos el archivo geojson
            sink.write({
                'properties': {
                    'rowid':i,
                    'cod_canton':cod_canton,
                    'canton':canton,
                    'calc_area':calc_area,
                    'calc_longitud':calc_longitud,
                    'calc_densidad_vial':densidad,
                    'calc_longitud_autopistas':calc_longitud_autopistas / 1000,
                    'calc_longitud_carr_pav_2_vias':calc_longitud_carr_pav_2_vias / 1000,
                    'calc_longitud_carr_pav_1_vias':calc_longitud_carr_pav_1_vias / 1000,
                    'calc_longitud_carr_nopav_2_vias':calc_longitud_carr_nopav_2_vias / 1000,
                    'calc_longitud_caminos_tierra':calc_longitud_caminos_tierra / 1000
                },
                'geometry':record_cant['geometry']
            }) 
            i += 1

# Ultimo paso. Se agrega el archivo GeoJSON de redvial-cantones al GPKG
print("Salvando archivo GPKG...")
with fiona.open('redvial-cantones.geojson') as source:
    with fiona.open('densidad-vial.gpkg', 'w', 'GPKG', source.schema, source.crs, layer='redvial-cantones') as sink:
        for record in source:
            sink.write(record)
#Fin.

Descargando capas de cantones y vias..
Creando archivo GPKG...
Calculando areas, longitudes e interescciones...
1 610 Corredores 623.6117592254898 495.8243217580492 0.7950849457583202
2 607 Golfito 1752.747561302697 602.6804356307817 0.3438489654396385
3 608 Coto Brus 944.2355403996279 747.302687151722 0.7914367286317584
4 605 Osa 1932.6972545987687 678.898892565084 0.3512701696810888
5 603 Buenos Aires 2382.939652165123 1297.0860451637627 0.5443218186349114
6 119 Pérez Zeledón 1901.082201518083 1429.5606320730353 0.7519720246349576
7 606 Quepos 557.8543756088676 399.1154483240083 0.7154473743947882
8 704 Talamanca 2792.2256992529396 328.3250452659486 0.11758542490092831
9 609 Parrita 483.2179953646667 339.89004373719166 0.7033886299716327
10 105 Tarrazú 291.2676390960465 295.72995222679384 1.0153203189499398
11 117 Dota 404.44367669572637 216.78212122823336 0.5360007677690168
12 611 Garabito 316.0105062089333 280.84730584736184 0.888727749012487
13 120 León Cortés Castro 121.894572774

In [21]:
# Tarea 3. Item 1. 
# Tabla de Cantones

import pandas as pd

print("Item 1. Tabla de Cantones.")
i = 0
list = []
with fiona.collection('densidad-vial.gpkg', 'r', layer='redvial-cantones') as cant:
    for record_cant in cant:
        canton = record_cant['properties']['canton']
        calc_longitud = record_cant['properties']['calc_longitud']
        densidad = record_cant['properties']['calc_densidad_vial']
        # Calculos adicionales de longitud
        calc_longitud_autopistas = record_cant['properties']['calc_longitud_autopistas']
        calc_longitud_carr_pav_2_vias = record_cant['properties']['calc_longitud_carr_pav_2_vias']
        calc_longitud_carr_pav_1_vias = record_cant['properties']['calc_longitud_carr_pav_1_vias']
        calc_longitud_carr_nopav_2_vias = record_cant['properties']['calc_longitud_carr_nopav_2_vias']
        calc_longitud_caminos_tierra = record_cant['properties']['calc_longitud_caminos_tierra']
        s1 = pd.Series([canton, calc_longitud, densidad, calc_longitud_autopistas, calc_longitud_carr_pav_2_vias,
                           calc_longitud_carr_pav_1_vias, calc_longitud_carr_nopav_2_vias, calc_longitud_caminos_tierra],
                      index = ["Canton", "Longitud Total", "Densidad Total", "Autopistas", "Pav 2 vias",  "Pav 1 via", "Sin Pav 2 vias", "Caminos Tirra"])
        list.append(s1)
tabla_cant = pd.DataFrame(list)
tabla_cant

Item 1. Tabla de Cantones.


,Canton,Longitud Total,Densidad Total,Autopistas,Pav 2 vias,Pav 1 via,Sin Pav 2 vias,Caminos Tirra
0,Corredores,495.824322,0.795085,0.0,130679.909911,99151.474216,12139.449335,253853.488296
1,Golfito,602.680436,0.343849,0.0,80781.497428,127669.233277,72046.380742,322183.324183
2,Coto Brus,747.302687,0.791437,0.0,108759.457131,369560.744862,0.000000,268982.485159
3,Osa,678.898893,0.351270,0.0,134506.070512,153449.033000,55998.104238,334945.684816
4,Buenos Aires,1297.086045,0.544322,0.0,170930.288013,434423.276523,0.000000,691732.480628
...,...,...,...,...,...,...,...,...
77,Guatuso,308.730568,0.410096,0.0,19648.570660,76574.170963,0.000000,212507.826831
78,Los Chiles,776.987780,0.583013,0.0,114471.867585,228786.537747,0.000000,433729.374306
79,Liberia,1070.443021,0.742245,0.0,39948.298007,278803.922916,0.000000,751690.800395
80,Upala,770.134477,0.483548,0.0,9263.710334,418678.392783,2904.216286,339288.157595
